In [1]:
import sys
sys.path.append('../30_data_tools/')
sys.path.append('../60_gebastel/process_masks//')
import torch
from PIL import ImageDraw

In [2]:
from render_masks import render_mask, get_config
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
from helper import load_dotenv, get_pdf_page_processing_status
from file_interaction import open_img, get_related_filepath

In [3]:
from mask_functions import is_below_max_size, is_above_min_size, is_text_mask, filter_intersected_masks

In [4]:
import cv2

In [5]:
pdf_filename = '617995.tg45_Inhalt_7128_BA_WG1_0159_D.p1'
pdf_filename = '617660.0065_FS_24_41-Dat.p1'

In [6]:
dotenv = load_dotenv()
config = get_config()
config['target_variant'] = 'halftone600dpi'

sam = sam_model_registry["vit_h"](checkpoint=dotenv['MODEL_DIR'] / "sam_vit_h_4b8939.pth")

if torch.cuda.is_available():
    sam.to(device=torch.cuda.device(0))
    print( torch.cuda.get_device_name(0) )
else:
    print( "no cuda" )

mask_generator = SamAutomaticMaskGenerator(sam)

no cuda


In [7]:
masks = get_pdf_page_processing_status( config['target_variant'], 'masks' )
masks = masks.loc[
    masks.job == '24-03-05-01_randomTrainPages'
]

In [8]:
row = masks.loc[
    (masks.job == '24-03-05-01_randomTrainPages') &
    (masks.filename == pdf_filename)
].iloc[0]

In [9]:
import numpy as np
from PIL import Image

In [10]:
found_masks = render_mask( row, mask_generator, config, use_filters=False )

In [28]:
img = open_img(
    get_related_filepath(
        row.job,
        'halftone600dpi',
        f'{row.filename}.4c.jpg'
    )
)
img = img.convert('RGBA')

In [19]:
img.convert('RGB').save('/Users/frederic.birwe/Documents/GitHub/ma_obsidian/Attachments/7_Maskenfilterung/seite_original.jpg')

In [12]:
below_max_size = [i for i in range(len(found_masks)) if is_below_max_size(found_masks[i]) == False]
above_min_size = [
    i for i in range(len(found_masks))
    if is_above_min_size(found_masks[i]) == False and i not in below_max_size
]
text_mask = [
    i for i in range(len(found_masks))
    if i not in below_max_size and i not in above_min_size and is_text_mask( img, found_masks[i])
]

not_intersected_masks = filter_intersected_masks([
    found_masks[i] for i in range(len(found_masks))
    if i not in below_max_size and i not in above_min_size and i not in text_mask
])

intersected_masks = [
    i for i in range(len(found_masks))
    if i not in below_max_size and i not in above_min_size and i not in text_mask and found_masks[i] not in not_intersected_masks
]

ok_masks = [
    i for i in range(len(found_masks))
    if i not in below_max_size + above_min_size + text_mask + intersected_masks
]

/var/folders/ns/b_3z05zx1bz71dzh3qcvtpyh0000gq/T/ipykernel_44495/1717245196.py:18: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if i not in below_max_size and i not in above_min_size and i not in text_mask and found_masks[i] not in not_intersected_masks


    contours, hierarchy = cv2.findContours(
        (m['mask'] * 255).astype('uint8'),  
        cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_NONE
    )
    #out = cv2.drawContours(np.ones(m['mask'].shape) * 255, contours, -1, 0, 6).astype('uint8')

In [42]:
img_out = Image.new(
    'RGBA',
    img.size,
    color=(255, 0, 0, 0)
)
for i in range(len(found_masks)):
    m = found_masks[i]

    if i in below_max_size:
        color = '#ffcc32' 
    elif i in above_min_size:
        color = '#e04d27'
    elif i in text_mask:
        color = '#3e5396'
    elif i in intersected_masks:
        color = '#562996'
    else:
        color = '#618b00'
    
    img_out.paste(Image.new(
        'RGB',
        (m['mask'].shape[1], m['mask'].shape[0]),
        color=color
    ), (m['bbox'][0],m['bbox'][1]), Image.fromarray((m['mask'] * 255).astype('uint8')))

In [43]:
img_out.save('/Users/frederic.birwe/Documents/GitHub/ma_obsidian/Attachments/7_Maskenfilterung/seite_masken.png','PNG')